In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import matplotlib.gridspec as gridspec
import ast
import sys
sys.path.append('machine-scientist/')
sys.path.append('machine-scientist/Prior/')
from mcmc import *
from parallel import *
from fit_prior import read_prior_par
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
def clean_index(dataframe):
    dataframe.set_index('Unnamed: 0', inplace=True)
    dataframe.index.name = None
    dataframe= dataframe.reset_index(drop=True)
    return dataframe

def add_bms_pred(dataframe, bms_trace, number_param):
    VARS = ['x1',]
    x = dn[[c for c in VARS]].copy()
    y=dataframe.noise

    if number_param==10:
        prior_par = read_prior_par('machine-scientist/Prior/final_prior_param_sq.named_equations.nv1.np10.2017-10-18 18:07:35.089658.dat')
    elif number_param==20:
        prior_par = read_prior_par('machine-scientist/Prior/final_prior_param_sq.named_equations.nv1.np20.maxs200.2024-05-10 162907.551306.dat')

    #mdl model
    minrow = bms_trace[bms_trace.H == min(bms_trace.H)].iloc[0]
    minH, minexpr, minparvals = minrow.H, minrow.expr, ast.literal_eval(minrow.parvals)

    t = Tree(
        variables=list(x.columns),
        parameters=['a%d' % i for i in range(number_param)],
        x=x, y=y,
        prior_par=prior_par,
        max_size=200,
        from_string=minexpr,
    )

    t.set_par_values(deepcopy(minparvals))

    dplot = deepcopy(dn)
    dplot['ybms'] = t.predict(x)

    return dplot
    
    

In [ ]:
#Read NN and BMS data
function='tanh' #tanh, leaky_ReLU
realizations=2
N=9
#sigmas=[sigma_y for sigma_y in np.arange(0,0.2,0.02)]
sigmas=[0.02, 0.04,0.06, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18, 0.20]


runid=0
NPAR=10 #10, 20
steps=50000
train_size=60

rmse_nn_train=[];rmse_nn_test=[]
rmse_mdl_train=[];rmse_mdl_test=[]

mae_nn_train=[];mae_nn_test=[]
mae_mdl_train=[];mae_mdl_test=[]

n_index=[];r_index=[];sigma_index=[]

#Put mae and rmse of each simulation (on nn and bms) in a dataframe
for sigma in sigmas:
    for r in range(realizations+1):
        #Read NN data
        if sigma==0.1:
            file_model='NN_no_overfit_' + function + '_sigma_' + '0.10' + '_r_' + str(r) + '.csv'
        else:
            file_model='NN_no_overfit_' + function + '_sigma_' + str(sigma) + '_r_' + str(r) + '.csv'
            
        model_d='../data/trained_nns/' + file_model
        d=pd.read_csv(model_d)

        for n in range(N+1):
            n_index.append(n);r_index.append(r);sigma_index.append(sigma)
            
            dn=d[d['rep']==n]
            dn=clean_index(dn)

            #Read BMS data
            if sigma==0.1:
                filename='BMS_'+function+'_n_'+str(n)+'_sigma_'+'0.10'+ '_r_' + str(r) + '_trace_'+str(steps)+'_prior_'+str(NPAR)+ '.csv'
            else:
                filename='BMS_'+function+'_n_'+str(n)+'_sigma_'+str(sigma)+ '_r_' + str(r) + '_trace_'+str(steps)+'_prior_'+str(NPAR)+ '.csv'
        
            trace=pd.read_csv('../data/MSTraces/' + filename, sep=';', header=None, names=['t', 'H', 'expr', 'parvals', 'kk1', 'kk2','kk3'])
            dplot=add_bms_pred(dn, trace, NPAR)

            #Errors
            rmse_nn_train_i=root_mean_squared_error(dplot.loc[:train_size-1]['ymodel'],dplot.loc[:train_size -1]['y'])
            rmse_nn_train.append(rmse_nn_train_i)
            
            rmse_nn_test_i=root_mean_squared_error(dplot.loc[train_size-1:]['ymodel'],dplot.loc[train_size -1:]['y'])
            rmse_nn_test.append(rmse_nn_test_i)

            mae_nn_train_i=mean_absolute_error(dplot.loc[:train_size-1]['ymodel'],dplot.loc[:train_size -1]['y'])
            mae_nn_train.append(mae_nn_train_i)
            
            mae_nn_test_i=mean_absolute_error(dplot.loc[train_size-1:]['ymodel'],dplot.loc[train_size -1:]['y'])
            mae_nn_test.append(mae_nn_test_i)
    
            rmse_mdl_i=mean_squared_error(dplot.ybms,dn.y)
            
            rmse_mdl_train_i=root_mean_squared_error(dplot.loc[:train_size-1]['ybms'],dn.loc[:train_size-1]['y'])
            rmse_mdl_train.append(rmse_mdl_train_i)
            
            rmse_mdl_test_i=root_mean_squared_error(dplot.loc[train_size-1:]['ybms'],dn.loc[train_size-1:]['y'])
            rmse_mdl_test.append(rmse_mdl_test_i)

            mae_mdl_train_i=mean_absolute_error(dplot.loc[:train_size-1]['ybms'],dplot.loc[:train_size -1]['y'])
            mae_mdl_train.append(mae_mdl_train_i)
            
            mae_mdl_test_i=mean_absolute_error(dplot.loc[train_size-1:]['ybms'],dplot.loc[train_size -1:]['y'])
            mae_mdl_test.append(mae_mdl_test_i)

errors_df=pd.DataFrame({'sigma':sigma_index, 'mae_nn_train':mae_nn_train, 'mae_nn_test':mae_nn_test, 'mae_mdl_train':mae_mdl_train, 
                        'mae_mdl_test':mae_mdl_test, 'rmse_nn_train':rmse_nn_train, 'rmse_nn_test': rmse_nn_test, 
                        'rmse_mdl_train':rmse_mdl_train, 'rmse_mdl_test': rmse_mdl_test, 'n':n_index, 'r': r_index})
display(errors_df)

In [ ]:
columns_errors=['mae_nn_train','mae_nn_test','mae_mdl_train','mae_mdl_test','rmse_nn_train','rmse_nn_test','rmse_mdl_train','rmse_mdl_test']

#Mean values of 30 (3*10) simulations
mean_errors_df=errors_df.groupby(['sigma'],as_index=False)[columns_errors].mean()
#display(mean_errors_df)


#Error
#1. For every sigma and every n, calculate means over the realizations
mean_n_over_r_df=errors_df.groupby(['n', 'sigma'],as_index=False)[columns_errors].mean()
#display(mean_n_over_r_df)

#2. Calculate stds over means of over realizations
std_n_over_r_df=mean_n_over_r_df.groupby(['sigma'],as_index=False)[columns_errors].std() 

#rename columns
std_n_over_r_df=std_n_over_r_df.rename(columns={column:'std_' + column for column in columns_errors})
display(std_n_over_r_df)


#3. sem: divide by sqare root of realizations
std_n_over_r_df[['sem_mae_nn_train','sem_mae_nn_test','sem_mae_mdl_train','sem_mae_mdl_test',\
                 'sem_rmse_nn_train','sem_rmse_nn_test','sem_rmse_mdl_train','sem_rmse_mdl_test' ]]=\
std_n_over_r_df[['std_mae_nn_train', 'std_mae_nn_test', 'std_mae_mdl_train', 'std_mae_mdl_test', 'std_rmse_nn_train',\
                 'std_rmse_nn_test','std_rmse_mdl_train','std_rmse_mdl_test' ]]/np.sqrt(realizations+1)

#sem_n_over_r_df=std_n_over_r_df[columns_errors]/np.sqrt(realizations+1)
display(std_n_over_r_df)

#rename columns
std_n_over_r_df=std_n_over_r_df.rename(columns={'mae_nn_train':  'std_mae_nn_train',  'mae_nn_test':  'std_mae_nn_test', 
                                            'mae_mdl_train': 'std_mae_mdl_train', 'mae_mdl_test': 'std_mae_mdl_test',
                                            'rmse_nn_train': 'std_rmse_nn_train', 'rmse_nn_test': 'std_rmse_nn_test' , 
                                            'rmse_mdl_train':'std_rmse_mdl_train','rmse_mdl_test':'std_rmse_mdl_test'})


#sem_n_over_r_df=sem_n_over_r_df.rename(columns={'mae_nn_train':  'sem_mae_nn_train',  'mae_nn_test':  'sem_mae_nn_test', 
#                                            'mae_mdl_train': 'sem_mae_mdl_train', 'sem_mae_mdl_test': 'sem_mae_mdl_test',
#                                            'rmse_nn_train': 'sem_rmse_nn_train', 'sem_rmse_nn_test': 'sem_rmse_nn_test' , 
#                                            'rmse_mdl_train':'sem_rmse_mdl_train','sem_rmse_mdl_test':'sem_rmse_mdl_test'})


errors_statistics_new=mean_errors_df.join(std_n_over_r_df.set_index('sigma'), on='sigma')
display(errors_statistics_new)

print('../data/errors_statistics_' + str(function) + '.csv')
errors_statistics_new.to_csv('../data/errors_statistics_' + str(function) + '.csv')

In [5]:
##INCOMPLETE!!

#Plot rmse
Extensions=['.png', '.pdf']

#Fonts and sizes                                                                                    
size_axis=7;size_ticks=6;size_title=5
line_w=1;marker_s=3 #width and marker size                                                          
m_size=6

#Define figure size in cm                                                                           
cm = 1/2.54 #convert inch to cm                                                                     
width = 8*cm; height=6*cm
rows=3;cols=10

width_panel = width*cols
height_panel= height*rows

fig=figure(figsize=(width_panel,height_panel), dpi=300)  
gs=gridspec.GridSpec(rows,cols)                                                                                                                         
gs.update(left=0.1,right=0.98,bottom=0.15,top=0.90,wspace=0.35,hspace=0.1) 

for row in range(len(rows)):
    print(row)
    for col in range(len(cols)): 
        print(col)

        ax_ij=plt.subplot(gs[i,j])
        plt.plot(dn['x1'], dn['ymodel'],linewidth=line_w, color='red', label='ann.  MSE_train= %.2E, MSE_test= %.2E' % ( MAE_nn_train, MAE_nn_test))

SyntaxError: incomplete input (2887671460.py, line 23)